In [7]:
import os
import pandas as pd
from PIL import Image
import torch
import kagglehub
from sklearn.metrics import accuracy_score

from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from torch.utils.data import DataLoader
from torchvision import transforms

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
path = kagglehub.dataset_download("alessiocorrado99/animals10")

print("Path to dataset files:", path)

100%|███████████████████████████████████████████████████████████████████████████████| 586M/586M [01:03<00:00, 9.63MB/s]

Extracting files...


Path to dataset files: C:\Users\nikio\.cache\kagglehub\datasets\alessiocorrado99\animals10\versions\2
